### Cleaning Data Scraped from News Sites

In [67]:
import pandas as pd 
import numpy as np

Collecting Urdu stopwords.

In [68]:
urdu_stopwords = []
with open('./Cleaning Data/stopwords-ur.txt', 'r') as f:
    for line in f:
        urdu_stopwords.append(line.strip())
        
urdu_stopwords = set(urdu_stopwords)

In [69]:
jang_data = pd.read_csv('./Scraped Data/jang_data.csv')
ary_data = pd.read_csv('./Scraped Data/ary_urdu_articles.csv')
dunya_data = pd.read_csv('./Scraped Data/dunya_urdu_articles.csv')
express_data = pd.read_csv('./Scraped Data/express_data.csv')

Combining the data and checking the gold labels.

In [70]:
jang_gold_labels = set(jang_data['gold_label'].tolist())
ary_gold_labels = set(ary_data['category'].tolist())
dunya_gold_labels = set(dunya_data['category'].tolist())
express_gold_labels = set(express_data['gold_label'].tolist())
standerd_gold_labels = ['Entertainment', 'Business', 'Sports', 'Science-Technology', 'International']

print(jang_gold_labels)
print(ary_gold_labels)
print(dunya_gold_labels)
print(express_gold_labels)
print(standerd_gold_labels)

{'world', 'business', 'sports', 'entertainment'}
{'Sports', 'Entertainment', 'Business', 'Technology', 'World'}
{'Sports', 'Entertainment', 'Business', 'Technology', 'World'}
{'world', 'sports', 'business', 'entertainment', 'science-technology'}
['Entertainment', 'Business', 'Sports', 'Science-Technology', 'International']


Updating individual news site labels to a consistent set of gold labels.

In [71]:
def jang_gold_label_change(x):
    if x == 'sports':
        return 'Sports'
    elif x == 'business':
        return 'Business'
    elif x == 'entertainment':
        return 'Entertainment'
    elif x == 'world':
        return 'International'
    else:
        return None
    
def ary_gold_label_change(x):
    if x == 'World':
        return 'International'
    elif x == 'Technology':
        return 'Science-Technology'
    else:
        return x

def dunya_gold_label_change(x):
    if x == 'World':
        return 'International'
    elif x == 'Technology':
        return 'Science-Technology'
    else:
        return x
    
def express_gold_label_change(x):
    if x == 'sports':
        return 'Sports'
    elif x == 'business':
        return 'Business'
    elif x == 'entertainment':
        return 'Entertainment'
    elif x == 'science-technology':
        return 'Science-Technology'
    elif x == 'world':
        return 'International'
    else:
        return None

In [72]:
jang_data_corrected = jang_data.copy()
jang_data_corrected['gold_label'] = jang_data_corrected['gold_label'].apply(jang_gold_label_change)

ary_data_corrected = ary_data.copy()
ary_data_corrected['category'] = ary_data_corrected['category'].apply(ary_gold_label_change)

dunya_data_corrected = dunya_data.copy()
dunya_data_corrected['category'] = dunya_data_corrected['category'].apply(dunya_gold_label_change)

express_data_corrected = express_data.copy()
express_data_corrected['gold_label'] = express_data_corrected['gold_label'].apply(express_gold_label_change)

jang_corrected_gold_labels = set(jang_data_corrected['gold_label'].tolist())
ary_corrected_gold_labels = set(ary_data_corrected['category'].tolist())
dunya_corrected_gold_labels = set(dunya_data_corrected['category'].tolist())
express_corrected_gold_labels = set(express_data_corrected['gold_label'].tolist())

jang = False
ary = False
dunya = False
express = False
for label1, label2, label3, label4 in zip(jang_corrected_gold_labels, ary_corrected_gold_labels, dunya_corrected_gold_labels, express_corrected_gold_labels):
    if label1 not in standerd_gold_labels:
        jang = True
    if label2 not in standerd_gold_labels:
        ary = True
    if label3 not in standerd_gold_labels:
        dunya = True
    if label4 not in standerd_gold_labels:
        express = True
        
if jang:
    print('Jang Data has some incorrect labels')
else:
    print('Jang Data has correct labels')
    
if ary:
    print('Ary Data has some incorrect labels')
else:
    print('Ary Data has correct labels')
    
if dunya:
    print('Dunya Data has some incorrect labels')
else:
    print('Dunya Data has correct labels')
    
if express:
    print('Express Data has some incorrect labels')
else:
    print('Express Data has correct labels')

Jang Data has correct labels
Ary Data has correct labels
Dunya Data has correct labels
Express Data has correct labels


Creating a combined Dataset of all of the individual article CSVs.

In [73]:
data = pd.DataFrame(columns=['title', 'link', 'content', 'gold_label'])
data = pd.concat([data, jang_data_corrected[['title', 'link', 'content', 'gold_label']]], ignore_index=True)
ary_data_corrected.rename(columns={'category': 'gold_label'}, inplace=True)
data = pd.concat([data, ary_data_corrected[['title', 'link', 'content', 'gold_label']]], ignore_index=True)
dunya_data_corrected.rename(columns={'category': 'gold_label'}, inplace=True)
data = pd.concat([data, dunya_data_corrected[['title', 'link', 'content', 'gold_label']]], ignore_index=True)
data = pd.concat([data, express_data_corrected[['title', 'link', 'content', 'gold_label']]], ignore_index=True)
data.head()

,title,link,content,gold_label
0,یشمہ گل رشتے والی آنٹیوں کے بے جا مطالبات سے ...,https://jang.com.pk/news/1409434,معروف اداکارہ یشمہ گل رشتے والی آنٹیوں کے بےج...,Entertainment
1,شادی کے دوران جان سے مارنے کی دھمکیاں دی گئی ت...,https://jang.com.pk/news/1409426,کیا آپ جانتے ہیں جب سیف علی خان اور کرینہ کپو...,Entertainment
2,’وہ پہلے دودھ والا تھا‘، اجے دیوگن کا اکشے کی ...,https://jang.com.pk/news/1409422,سپر اسٹار اداکار اجے دیوگن نے ساتھی اداکار اکش...,Entertainment
3,کنگنا رناوت کی نانی چل بسیں، اداکارہ نے پرانی ...,https://jang.com.pk/news/1409416,بھارتی اداکارہ کنگنا رناوت کی نانی، اندرانی ٹھ...,Entertainment
4,بھارت میں جسے ناچ گانا نہیں آتا اس کا رشتہ نہ...,https://jang.com.pk/news/1409392,پاکستانی شوبز انڈسٹری سے وابستہ ماضی کی معروف ...,Entertainment


Dropping missing values.

In [74]:
missing_value_count = data.isnull().sum()
print("Missing Values in Data", missing_value_count)

Missing Values in Data title          0
link           0
content       24
gold_label     0
dtype: int64


In [75]:
data = data.dropna()
data

,title,link,content,gold_label
0,یشمہ گل رشتے والی آنٹیوں کے بے جا مطالبات سے ...,https://jang.com.pk/news/1409434,معروف اداکارہ یشمہ گل رشتے والی آنٹیوں کے بےج...,Entertainment
1,شادی کے دوران جان سے مارنے کی دھمکیاں دی گئی ت...,https://jang.com.pk/news/1409426,کیا آپ جانتے ہیں جب سیف علی خان اور کرینہ کپو...,Entertainment
2,’وہ پہلے دودھ والا تھا‘، اجے دیوگن کا اکشے کی ...,https://jang.com.pk/news/1409422,سپر اسٹار اداکار اجے دیوگن نے ساتھی اداکار اکش...,Entertainment
3,کنگنا رناوت کی نانی چل بسیں، اداکارہ نے پرانی ...,https://jang.com.pk/news/1409416,بھارتی اداکارہ کنگنا رناوت کی نانی، اندرانی ٹھ...,Entertainment
4,بھارت میں جسے ناچ گانا نہیں آتا اس کا رشتہ نہ...,https://jang.com.pk/news/1409392,پاکستانی شوبز انڈسٹری سے وابستہ ماضی کی معروف ...,Entertainment
...,...,...,...,...
1632,امریکی صدارتی الیکشن؛ 7 اہم ریاستوں میں کون سی...,https://www.express.pk/story/2731629/uselectio...,امریکا کے صدارتی انتخاب میں سوئنگ اسٹیٹس کہلائ...,International
1633,صدر منتخب ہونے پر بغیر دستاویز آنے والوں کو ڈی...,https://www.express.pk/story/2731602/sdr-mntkh...,امریکی صدارتی امیدوار ڈونلڈ ٹرمپ کا کہنا ہے کہ...,International
1634,امریکی انتخابات 2024، قبل از وقت ووٹوں کی تعدا...,https://www.express.pk/story/2731592/us-electi...,امریکی صدارتی انتخابات کے لیے کروڑوں امریکی ان...,International
1635,افغان ناظم الامور سے بلااجازت ملاقات، مالدیپ ن...,https://www.express.pk/story/2731591/afghan-na...,مالدیپ نے افغان ناظم الامور سے غیرسرکاری ملاقا...,International


Removing Urdu stopwords.

In [76]:
for content in data['content']:
    new_content = ''
    for word in content.split():
        if word in urdu_stopwords:
            continue
        else:
            new_content += word + ' '
    data['content'] = data['content'].replace(content, new_content)

Removing duplicates and adding article IDs.

In [77]:
data = data.drop_duplicates(subset='content', keep='first')
data.insert(0, "Article ID", range(1, len(data) + 1))

Removing HTML tags.

In [78]:
import re
for content in data['content']:
    new_content = ''
    for word in content.split():
        # Removing HTML Tags
        if word[0] == '<' and word[-1] == '>':
            continue
        elif re.match(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', word):
            continue
        elif re.match(r'\S+@\S+', word):
            continue
        new_content += word + ' '
    data['content'] = data['content'].replace(content, new_content)

/var/folders/h1/49stt73n6n7ch55lsp5m2zsc0000gn/T/ipykernel_34121/4217275969.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['content'] = data['content'].replace(content, new_content)


Removing non-Urdu words and characters.

In [ ]:
def clean_urdu_content(text, remove_numbers = True):
    urdu_only_pattern = r"[^\u0600-\u06FF\s]"
    if remove_numbers:
        urdu_only_pattern = r"[^\u0600-\u06FF\s]"
    else:
        urdu_only_pattern = r"[^\u0600-\u06FF\s0-9]"

    return re.sub(urdu_only_pattern, "", text)

data["content"] = data["content"].apply(lambda x: clean_urdu_content(x, remove_numbers = True))
data["title"] = data["title"].apply(lambda x: clean_urdu_content(x, remove_numbers = True))

# data.head()

Renaming titles.

In [81]:
mapping = {"title": "Titles", "link": "Links", "content": "Contents", "gold_label": "Gold Labels"}
data.reset_index(drop = True, inplace = True)
data.rename(columns = mapping, inplace = True)

data.to_csv('./Cleaning Data/cleaned_urdu_articles.csv', index = True)
data.sample(5)

# print(data["Gold Labels"].unique())

/var/folders/h1/49stt73n6n7ch55lsp5m2zsc0000gn/T/ipykernel_34121/3221932874.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns = mapping, inplace = True)


,Article ID,Titles,Links,Contents,Gold Labels
1499,1500,انتہائی پائیدار اسمارٹ فون,https://www.express.pk/story/2731682/techno-sp...,فون خریدتے وقت صارفین پائیداری اہم جز ہے۔چونکہ...,Science-Technology
1402,1403,زمین کی تقسیم در تقسیم زرعی پیداوار کو متاثر ک...,https://www.express.pk/story/2731411/zameen-ki...,زمین تقسیم ذرعی زمینوں پائیداری کو متاثر ہے، ج...,Business
458,459,آسٹریلیا کیخلاف پہلے ون ڈے کیلیے پاکستانی الی...,https://urdu.arynews.tv/pakistan-vs-australia-...,پاکستان آسٹریلیا خلاف تین ون ڈے میچوں سیریز س...,Sports
1251,1252,تھریڈز کے سوا سال میں صارفین کی تعداد ساڑھے ک...,https://urdu.dunyanews.tv/index.php/ur/Technol...,کیلی فورنیاویب ڈیسک میٹا نے جولائی کو ایکس ...,Science-Technology
1234,1235,سنیتا مارشل کا بطور والدہ بچوں کی تربیت کے حوا...,https://urdu.dunyanews.tv/index.php/ur/Enterta...,لاہور ویب ڈیسک معروف پاکستانی اداکارہ ماڈل سنی...,Entertainment
